In [4]:
# Required imports
import os
import numpy as np
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, add
import pickle
BASE_DIR = '/kaggle/input/flickr8k'
WORKING_DIR = '/kaggle/working'

# # load features from pickle
# with open(os.path.join(WORKING_DIR, '/kaggle/working/features.pkl'), 'rb') as f:
#     features = pickle.load(f)

In [18]:
def get_features():
    features = {} # Map to store the features
    for img in os.listdir(os.path.join(BASE_DIR, 'Images')):
        image = load_img(os.path.join(BASE_DIR, 'Images') + '/' + img, target_size = (224, 224))
        image = img_to_array(image)
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        image = preprocess_input(image)
        feature = model.predict(image, verbose = 0)
        image_id = img.split('.')[0]
        features[image_id] = feature  
    pickle.dump(features, open(os.path.join(WORKING_DIR, 'features.pkl'), 'wb')) # dumping features in the pickle.
    return features

In [19]:
model = VGG16()
model = Model(inputs = model.inputs, outputs = model.layers[-2].output)

In [20]:
features = get_features() # get the image features by running them through CNN.

In [16]:
# load features from pickle
with open(os.path.join(WORKING_DIR, '/kaggle/working/features.pkl'), 'rb') as f:
    features = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/features.pkl'

In [9]:
def get_train_data():
    dir = '/kaggle/input/some-files'
    new_features = {}
    with open(os.path.join(dir, 'Flickr_8k.trainImages.txt'), 'r') as f:
        next(f)
        captions_doc = f.read()
    for line in captions_doc.split('\n'):
        new_features[line] = features[line]
        
    return new_features
    
    
def get_test_data():
    dir = '/kaggle/input/some-files/Flickr_8k.valImages.txt'
    
    
# train_features = get_train_data()

In [13]:
with open(os.path.join(BASE_DIR, 'captions.txt'), 'r') as f:
    next(f)
    captions_doc = f.read()

mapping = {}

for line in captions_doc.split('\n'):    
    tokens = line.split(',')
    if len(line) < 2:
        continue
    image_id, caption = tokens[0], tokens[1:]    
    image_id = image_id.split('.')[0]    
    caption = " ".join(caption)
    if image_id not in mapping:
        mapping[image_id] = []        
    mapping[image_id].append(caption)

  0%|          | 0/40456 [00:00<?, ?it/s]

In [15]:
def clean(mapping):
    for key, captions in mapping.items():
        for i in range(len(captions)):
            caption = captions[i]
            caption = caption.lower()
            caption = caption.replace('[^A-Za-z]', '')
            caption = caption.replace('\s+', ' ')
            caption = 'startseq ' + " ".join([word for word in caption.split() if len(word)>1]) + ' endseq'
            captions[i] = caption


clean(mapping) # preprocessing.

In [19]:
all_captions = []
for key in mapping:
    for caption in mapping[key]:
        all_captions.append(caption)

In [20]:
len(all_captions)

40455

In [25]:
# tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_captions)
vocab_size = len(tokenizer.word_index) + 1
vocab_size
max_length = max(len(caption.split()) for caption in all_captions)

NameError: name 'all_captions' is not defined

In [24]:
image_ids = list(mapping.keys())
split = int(len(image_ids) * 0.90)
train = image_ids[:split]
test = image_ids[split:]

In [25]:
def data_generator(data_keys, mapping, features, tokenizer, max_length, vocab_size, batch_size):
    X1, X2, y = list(), list(), list()
    n = 0
    
    while 1:
        for key in data_keys:
            n += 1
            captions = mapping[key]            
            for caption in captions:
                seq = tokenizer.texts_to_sequences([caption])[0]                
                for i in range(1, len(seq)):                    
                    in_seq, out_seq = seq[:i], seq[i]
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]                    
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]                    
                    X1.append(features[key][0])
                    X2.append(in_seq)
                    y.append(out_seq)
                    
            if n == batch_size:
                X1, X2, y = np.array(X1), np.array(X2), np.array(y)
                yield [X1, X2], y
                X1, X2, y = list(), list(), list()
                n = 0

In [26]:

inputs1 = Input(shape=(4096,))
fe1 = Dropout(0.4)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)
inputs2 = Input(shape=(max_length,))
se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
se2 = Dropout(0.4)(se1)
se3 = LSTM(256)(se2)
decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)
model = Model(inputs=[inputs1, inputs2], outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [28]:
epochs = 20
batch_size = 32
steps = len(train) // batch_size

for i in range(epochs):
    generator = data_generator(train, mapping, features, tokenizer, max_length, vocab_size, batch_size)
    model.fit(generator, epochs=1, steps_per_epoch=steps, verbose=1)
    
model.save(WORKING_DIR+'/best_model.h5')

227/227 [==============================] - 63s 279ms/step - loss: 2.1866


In [30]:
def idx_to_word(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

In [31]:
def predict_caption(model, image, tokenizer, max_length):   
    in_text = 'startseq'
    
    for i in range(max_length):
        # encode input sequence
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        
        sequence = pad_sequences([sequence], max_length)
        yhat = model.predict([image, sequence], verbose=0)
        yhat = np.argmax(yhat)
        
        word = idx_to_word(yhat, tokenizer)
        if word is None:
            break
        in_text += " " + word
        if word == 'endseq':
            break
      
    return in_text

In [32]:
from nltk.translate.bleu_score import corpus_bleu
actual, predicted = list(), list()

for key in tqdm(test):
    
    captions = mapping[key]
    y_pred = predict_caption(model, features[key], tokenizer, max_length)
    actual_captions = [caption.split() for caption in captions]
    y_pred = y_pred.split()
    actual.append(actual_captions) 
    predicted.append(y_pred)
    
# calcuate BLEU score
print("BLEU: %f" % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))


  0%|          | 0/810 [00:00<?, ?it/s]

BLEU: 0.543720


In [33]:
from PIL import Image
import matplotlib.pyplot as plt

def generate_caption(image_name):
    
    # load the image
    image_id = image_name.split('.')[0]
    img_path = os.path.join(BASE_DIR, "Images", image_name)
    image = Image.open(img_path)
    captions = mapping[image_id]
    for caption in captions:
        print(caption)
        
    # predict the caption
    y_pred = predict_caption(model, features[image_id], tokenizer, max_length)
    print(y_pred)
    plt.imshow(image)